# Marketplace Conversion Funnel Analysis

## Overview
This notebook performs comprehensive analysis of the marketplace conversion funnel from search to payment, using SQL for data processing and Python for visualization and insights.

### Business Questions
- What is the overall conversion rate from search to payment?
- Where are users dropping off in the funnel?
- What factors influence conversion rates?
- How can we improve conversion rates?

### Data Sources
1. **all_search_events**: Search activity data
2. **view_listing_detail_events**: Listing view data  
3. **reservations**: Booking and payment data
4. **amplitude_user_ids**: User ID mapping


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Libraries imported successfully!")


## Data Loading and Database Setup

First, we'll load the CSV files and create a SQLite database for efficient SQL analysis.


In [ ]:
# Load CSV files and create SQLite database
print("Loading CSV files...")

# Load the datasets
search_events = pd.read_csv('all_search_events (1).csv')
listing_views = pd.read_csv('view_listing_detail_events (1).csv')
reservations = pd.read_csv('reservations (1).csv')
user_ids = pd.read_csv('amplitude_user_ids (1).csv')

print(f"Search events: {len(search_events):,} records")
print(f"Listing views: {len(listing_views):,} records")
print(f"Reservations: {len(reservations):,} records")
print(f"User IDs: {len(user_ids):,} records")

# Convert date columns
search_events['event_time'] = pd.to_datetime(search_events['event_time'])
search_events['event_date'] = pd.to_datetime(search_events['event_date'])

listing_views['event_time'] = pd.to_datetime(listing_views['event_time'])
listing_views['event_date'] = pd.to_datetime(listing_views['event_date'])

reservations['created_at'] = pd.to_datetime(reservations['created_at'])
reservations['approved_at'] = pd.to_datetime(reservations['approved_at'])
reservations['successful_payment_collected_at'] = pd.to_datetime(reservations['successful_payment_collected_at'])

print("Date columns converted successfully!")


In [ ]:
# Create SQLite database for SQL analysis
print("Creating SQLite database...")

conn = sqlite3.connect('marketplace_analysis.db')

# Write data to database
search_events.to_sql('search_events', conn, if_exists='replace', index=False)
listing_views.to_sql('listing_views', conn, if_exists='replace', index=False)
reservations.to_sql('reservations', conn, if_exists='replace', index=False)
user_ids.to_sql('amplitude_user_ids', conn, if_exists='replace', index=False)

print("Database created successfully!")
print("Tables created: search_events, listing_views, reservations, amplitude_user_ids")


## SQL Analysis - Conversion Funnel

Now we'll use SQL to perform comprehensive analysis of the conversion funnel. This approach allows us to handle large datasets efficiently and perform complex joins and aggregations.
